# Cincinnati Blight - Model Selection 
-----


In [ ]:
import pandas as pd
import numpy as np
import sklearn
import os
import fnmatch

# Config & database
from sqlalchemy import create_engine
import yaml
import pymongo
from pymongo import MongoClient

#import dateparser
from datetime import datetime
from datetime import date

from lib_cinci.evaluation import (load_one_inspection_per_parcel, 
                                  add_latlong_to_df)

# for get_best_from_experiment
import re


### Specify your parameters here.

- `experiment_directory`: Location of experiments you want to compare.

In [ ]:
experiment_directory = os.path.join(os.getcwd(), 'gridrun', 
                                   'splits', 'medium_models')

- `validation_start`, `validation_end`: The validation window start and end dates, in the format `'Dec 31 2015'`. Possible values for `validation_start` are `'Jun 30 2014'`, `'Dec 31 2014'`, `'Jun 30 2015'`, `'Dec 31 2015'`, `'Aug 31 2016'`. In general we use a 6 month validation window.

In [ ]:
validation_start = 'Dec 31 2015'
validation_end = 'Jun 30 2016'

- `space_delta`, `time_delta`: Level of spatiotemporal aggregation to consider for neighborhood history.

`space_delta` can take on values of `200m`, `400m`, `700m`, or `1000m`; `time_delta` can take on values of `3months`, `6months`, or `12months`. 

In [ ]:
space_delta = '400m'
time_delta = '12months'

- `results_filepath`: Where to store the CSV of model results.

In [ ]:
results_filepath = 'model-results-' + str(date.today()) + '.csv'

- `k`: Summary statistics (e.g. the average neighborhood inspection density or violation rate) are calculated on the "top *k*" parcels. 

In [ ]:
# 5% of all parcels 
k = 7500

## Setup

In [ ]:
# Configuration and DB connection
from sklearn_evaluation.Logger import Logger
folder = os.environ['ROOT_FOLDER']
name = 'config.yaml'
path = "%s/%s" % (folder, name)
f = open(path, 'r')
text = f.read()
main = yaml.load(text)

def load(name):
    folder = os.environ['ROOT_FOLDER']
    path = "%s/%s" % (folder, name)
    with open(path, 'r') as f:
        text = f.read()
    dic = yaml.load(text)
    return dic

connparams = load('config.yaml')['db']
uri = '{dialect}://{user}:{password}@{host}:{port}/{database}'.format(**connparams)
libpq_uri = 'dbname={database} user={user} host={host} password={password} port={port}'.format(**connparams)


engine = create_engine(uri)
logger = Logger(host=main['logger']['uri'], db=main['logger']['db'], 
                collection=main['logger']['collection'])

## Get experiments

Enter the directory where the config files you want to explore are located. All the config files in that directory and directories below will be included.

In [ ]:
experiment_configs = []

for root, dirnames, filenames in os.walk(experiment_directory):
    for filename in fnmatch.filter(filenames, '*.yaml'):
        experiment_configs.append(os.path.join(root, filename))


In [ ]:
def get_config_parameters(experiment_config):
    with open(experiment_config, 'r') as f:
        df = pd.io.json.json_normalize(yaml.load(f))
        df.set_index('experiment_name', drop=False, inplace=True)
        return df

In [ ]:
experiments = {experiment_config: get_config_parameters(experiment_config) 
     for experiment_config in experiment_configs}

In [ ]:
experiment_names = map(lambda v : v['experiment_name'][0], experiments.values())

In [ ]:
all_models_list = [logger.get_all_from_experiment(exp_name) for exp_name in  experiment_names]

In [ ]:
all_models = [item for sublist in all_models_list for item in sublist]

In [ ]:
for m in all_models:
    m['model_id'] = str(m['_id'])

In [ ]:
all_models_df = pd.DataFrame(all_models)
all_models_df.to_csv('all_models.csv', index=False)

## Get validation scores and results
-----

Load validation inspections (all inspections between validation start date and validation end date) to calculate precision on validation set.


In [ ]:
f = '%d%b%Y'

validation_feature_table = 'features_' + datetime.strptime(validation_start, '%b %d %Y').strftime(f).lower()

In [ ]:
#load results/labels for actual inspections between validation_start and validation_end
f = '%Y-%m-%d'

start = datetime.strptime(validation_start, '%b %d %Y').strftime(f)
end = datetime.strptime(validation_end, '%b %d %Y').strftime(f)

# load inspection results from validation window
validation_inspections = load_one_inspection_per_parcel(start, end)

In [ ]:
validation_inspections.reset_index(inplace=True)

In [ ]:
validation_inspections.to_csv('validation_inspections.csv')

In [ ]:
# get predictions on top k parcels for validation start date  
output_folder = os.environ['OUTPUT_FOLDER']
path_to_predictions = os.path.join(output_folder, 'top_predictions_on_all_parcels')

Here we retrieve scores for *all* parcels in Cincinnati at the beginning of the validation window. These are calculated and saved during the model run (`cincinnati/model/model.py`) using `validation_feature_table`, which has features for all parcels as of `validation_start` (6 months after the end of the training window).

In [ ]:
def get_model_precision(m, k=k): 
    
    model_top_k = pd.read_csv(os.path.join(path_to_predictions, m['model_id']), 
                              nrows=k+1, usecols=['parcel_id', 'prediction']) 
    
    top_k_inspected = validation_inspections[validation_inspections['parcel_id'].isin(model_top_k['parcel_id'])]
    
    # Calculate metrics on validation window
    validation_precision_at_p =  100.0*top_k_inspected.viol_outcome.sum()/top_k_inspected.shape[0]
    
    # Get percent of labeled data (inspected parcels) in validation set 
    validation_labeled_percent = 100.0*(top_k_inspected.shape[0])/k
    
    return validation_precision_at_p, validation_labeled_percent

In [ ]:
for m in all_models:
    m['validation_precision_at_p'], m['validation_labeled_percent'] = get_model_precision(m)

## Neighborhood inspection and violation history
-----

In [ ]:
validation_start = 'Aug 31 2016'

In [ ]:
neighborhood_table = 'neighborhood_score_' + space_delta + '_' + time_delta

f = '%d%b%Y'

date_tag = datetime.strptime(validation_start, '%b %d %Y').strftime(f).lower()
validation_feature_table = 'features_' + date_tag


In [ ]:
neighborhood_history = pd.read_sql_table(neighborhood_table, engine, 
                                             schema = validation_feature_table,
                                                index_col=['parcel_id'])

In [ ]:
# Calculate number of unique inspections per house and rate of violation rate
# in each neighborhood
neighborhood_history['violations_per_house'] = neighborhood_history.unique_violations/neighborhood_history.houses
neighborhood_history['violations_per_inspection'] = neighborhood_history.unique_violations/neighborhood_history.unique_inspections
neighborhood_history['inspection_density'] = neighborhood_history.unique_inspections/neighborhood_history.houses
neighborhood_history.sort_values(by='violations_per_house', ascending=False, inplace=True)

neighborhood_history.drop('inspection_date', axis=1, inplace=True)
    

In [ ]:
neighborhood_with_location = add_latlong_to_df(neighborhood_history[['violations_per_house', 'violations_per_inspection','inspection_density']])
neighborhood_with_location.to_csv('parcels_with_neighborhood_info_' + date_tag + '.csv')

In [ ]:
inspection_density_first_quartile = neighborhood_history['inspection_density'].quantile(0.25)
violations_per_house_first_quartile = neighborhood_history['violations_per_house'].quantile(0.25)
violations_per_inspection_first_quartile = neighborhood_history['violations_per_inspection'].quantile(0.25)

- `top_10_inspection_density_mean`, `top_10_inspection_density_std_dev`: The mean and standard deviation of inspection density for the top 10% of parcels, where parcels are ranked by this model's scores. 
- `top_10_violation_rate_mean`, `top_10_violation_rate_std_dev`: The mean and standard deviation of violation rate for the top 10% of parcels, where parcels are ranked by this model's scores. 
- `top_10_non_null_houses_percent`: The percent of parcels (in this model's top 10%) that have at least 1 other house in their "neighborhood."
- `top_10_low_inspection_density_percent`: The percent of parcels (in this model's top 10%) that have neighborhood inspection densities in the bottom 25% of all neighborhood inspection densities.
- `top_10_low_violation_rate_percent`: The percent of parcels (in this model's top 10%) that have neighborhood violation rates in the bottom 25% of all neighborhood inspection densities.

# Save Results to CSV 
----

In [ ]:
all_models_df = pd.DataFrame(all_models).drop('_id', axis=1)

In [ ]:
all_models_df.set_index('model_id', inplace=True)

In [ ]:
all_models_df['prec_std_dev_across_p'] = all_models_df[['prec_at_1', 
                                                        'prec_at_5', 
                                                        'prec_at_10', 
                                                        'prec_at_20']].apply(np.std, axis=1)

In [ ]:
# save results to CSV 
all_models_df.to_csv(results_filepath)